In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

In [2]:
directory_in = './file/1.item_number'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：512



,No,Item Number
0,1,255410646663
1,2,335270737835
2,3,232315264904
3,4,235748987359
4,5,252900220689
...,...,...
507,508,335624941617
508,509,234276860559
509,510,334151851057
510,511,234544398159


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                        1
 Item Number    255410646663
 Name: 0, dtype: object)

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = f'''https://www.ebay.com/itm/{crawler_series['Item Number']}?_ul=US&_stpos=91710&orig_cvip=true'''

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

            # = = = = = = = = = = = = = = =

            if dict_['o']['g']['itemId'].strip() == crawler_series['Item Number']:
                with open('./url.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))

                list_dict = [list_[2] for list_ in dict_['o']['w'] if 'model' in list_[2]]

                # = = = = = = = = = = = = = = =

                for link in list_dict[0]['model']['modules']['MARS']['metaData']:
                    soup = BeautifulSoup(link, 'lxml')
                    html = etree.HTML(str(soup))

                    # = = = = = = = = = = = = = = =

                    if html is not None and html.xpath('//link[@hreflang="x-default"]'):
                        url = html.xpath('//link/@href')[0]

                        if url.startswith('https://www.ebay.com/'):
                            url = f'{url}?_ul=US&_stpos=91710&orig_cvip=true'
                        elif url.startswith('https://www.ebay.de/'):
                            url = f'{url}?_ul=DE&_stpos=10115&orig_cvip=true'
                        elif url.startswith('https://www.ebay.co.uk/'):
                            url = f'{url}?_ul=GB&_stpos=BS81QU&orig_cvip=true'
                        elif url.startswith('https://www.ebay.com.au/'):
                            url = f'{url}?_ul=AU&_stpos=2019&orig_cvip=true'
                        elif url.startswith('https://www.ebay.ca/'):
                            url = f'{url}?_ul=CA&_stpos=M5S2E8&orig_cvip=true'

                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp.status_code

1
2


200

In [6]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Item Number': crawler_series['Item Number'],
                          'Url': url})

list_dict_correct

[{'No': 1,
  'Item Number': '255410646663',
  'Url': 'https://www.ebay.com/itm/255410646663?_ul=US&_stpos=91710&orig_cvip=true'}]

In [7]:
crawler_status = True

crawler_status

True

In [8]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Item Number                                         255410646663
 Request_Url    https://www.ebay.com/itm/255410646663?_ul=US&_...
 Name: 0, dtype: object]

In [9]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_url.xlsx', index=False)

df_correct

,No,Item Number,Url
0,1,255410646663,https://www.ebay.com/itm/255410646663?_ul=US&_...


In [10]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_url_error.xlsx', index=False)

df_error

,No,Item Number,Request_Url
0,1,255410646663,https://www.ebay.com/itm/255410646663?_ul=US&_...
